## Grocery Sales Prediction using Machine Learning 🛒

>Prakruthi Sidram



### Loading Dataset 🔢

In [ ]:
# Libraries 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display= 'diagram')

In [ ]:
# data
path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQPMdCEJ1LpPZar8BrFWWWcZ0hC-77-3LtM8O02UAp-hQ8fuXWVd_NDf5zeKu1nYVS1D6HtrgzemDMS/pub?output=csv'

In [ ]:
grocery = pd.read_csv(path)
grocery.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
# Copy the dataframe
df = grocery.copy()

####  Initial Inspection 🔎

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
# Duplicates Check and Fix

df.duplicated().sum()

0

In [ ]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Our '`target`' for this model is `Item_Outlet_Sales`
which does not have any `missing` values. So we will not `drop` any rows at this point.

Exploring the Features 🔍

 Columns | Dtype | Feature Type
--------- | --------| ----------| 
0   Item_Identifier   |    object | Nominal 
1   Item_Weight  |   float64 | Numerical
2   Item_Fat_Content    |  object | Nominal
3   Item_Visibility         | float64 | Numerical
4   Item_Type               | object | Nominal
5   Item_MRP                   |  float64 | Numerical
6   Outlet_Identifier          |  object | Nominal
7  Outlet_Establishment_Year   |int64 | Numerical
8   Outlet_Size                  |object | Ordinal
9   Outlet_Location_Type         |object | Ordinal
10  Outlet_Type                 |object | Nominal
11  Item_Outlet_Sales            |float64| Numerical

#### Pre Split Cleaning
Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (There is a way to do this after the split, but for this project, you may perform this step before the split)

In [ ]:
# Dropping the Identifier Columns as it won't help the model
df.drop(['item_identifier','outlet_identifier'], axis=1,inplace=True)

In [ ]:
for col in df.select_dtypes(include= 'object').columns:
  display(df[col].value_counts())
  print('\n')

Low Fat    5517
Regular    3006
Name: item_fat_content, dtype: int64

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: item_type, dtype: int64

Medium    2793
Small     2388
High       932
Name: outlet_size, dtype: int64

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: outlet_location_type, dtype: int64

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: outlet_type, dtype: int64

In [ ]:
# Fixing Fat Content Column 
df.columns = df.columns.str.lower()
fat_content_map = {'reg':'Regular',
                   'LF' : 'Low Fat',
                   'low fat' : 'Low Fat'
                   }
df['item_fat_content'] = df['item_fat_content'].replace(fat_content_map)

In [ ]:
df['item_fat_content'].value_counts()

Low Fat    5517
Regular    3006
Name: item_fat_content, dtype: int64

In [ ]:
df.replace(['High'],['Large'],inplace=True)

In [ ]:
df['outlet_size'].value_counts()

Medium    2793
Small     2388
Large      932
Name: outlet_size, dtype: int64

### Validation Split 🔪

In [ ]:
# Target y :
y = df['item_outlet_sales']
# Features X
X = df.drop(columns=['item_outlet_sales'])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

### Pipelines & Transformers *🗒*

#### Instantiate Transformers 🖌

In [ ]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown= 'ignore')

mean_imputer = SimpleImputer(strategy='mean')
most_freq_imputer = SimpleImputer(strategy='most_frequent')


##### Ordinal Encoder

In [ ]:
df['outlet_location_type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: outlet_location_type, dtype: int64

In [ ]:
# Ordinal Category : outlet_size , outlet_loc_type
outletsize_labels = ['Small','Medium','Large']
outletloctype_labels =['Tier 3','Tier 2','Tier 1']

ordered_labels = [outletsize_labels, outletloctype_labels]

ordinal = OrdinalEncoder(categories=ordered_labels)

#### Pipelines

In [ ]:
number_pipe = make_pipeline(mean_imputer,scaler)
ordinal_pipe = make_pipeline(most_freq_imputer,ordinal)
nominal_pipe = make_pipeline(most_freq_imputer,ohe)

In [ ]:
# tuples to pair pipeline with columns

num_selector = make_column_selector(dtype_include='number')
ord_cols = ['outlet_size','outlet_location_type']
nom_cols = ['item_fat_content', 'item_type','outlet_type']

num_tuple = (number_pipe, num_selector)
ord_tuple = (ordinal_pipe, ord_cols)
nom_tuple = (nominal_pipe,nom_cols)

#### Applying  preprocessing to columns


In [ ]:
preprocessor = make_column_transformer(ord_tuple,
                                       nom_tuple,
                                       num_tuple,
                                       remainder = 'passthrough')

In [ ]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'Large'],
                                                                             ['Tier '
                                                                              '3',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '1']]))]),
                                 ['outlet_size', 'outlet_location_type']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['item_fat_content', 'item_type',
                                  'outlet_type']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f0849677ed0>)])

#### Fit Transformer on the Training Data

In [ ]:
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'Large'],
                                                                             ['Tier '
                                                                              '3',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '1']]))]),
                                 ['outlet_size', 'outlet_location_type']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['item_fat_content', 'item_type',
                                  'outlet_type']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f0849677ed0>)])

In [ ]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
X_train_df = pd.DataFrame(X_train_processed)
display(X_train_df.head())
X_train_df.info()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.817249,-0.712775,1.828109,1.327849
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.556340,-1.291052,0.603369,1.327849
2,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.131512,1.813319,0.244541,0.136187
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.169219,-1.004931,-0.952591,0.732018
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.528819,-0.965484,-0.336460,0.493686


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6392 entries, 0 to 6391
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6392 non-null   float64
 1   1       6392 non-null   float64
 2   2       6392 non-null   float64
 3   3       6392 non-null   float64
 4   4       6392 non-null   float64
 5   5       6392 non-null   float64
 6   6       6392 non-null   float64
 7   7       6392 non-null   float64
 8   8       6392 non-null   float64
 9   9       6392 non-null   float64
 10  10      6392 non-null   float64
 11  11      6392 non-null   float64
 12  12      6392 non-null   float64
 13  13      6392 non-null   float64
 14  14      6392 non-null   float64
 15  15      6392 non-null   float64
 16  16      6392 non-null   float64
 17  17      6392 non-null   float64
 18  18      6392 non-null   float64
 19  19      6392 non-null   float64
 20  20      6392 non-null   float64
 21  21      6392 non-null   float64
 22  